<a href="https://colab.research.google.com/github/Gabriel-PC/DIO_Bootcamp-DS/blob/main/PipeLine_de_ETL/PipeLine_de_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Guia Rápido desta pipeline

Encontre o Relatório: Visite o site da OCDE e encontre o relatório anual de economia.

Download Manual: Faça o download do relatório manualmente para uma pasta em seu sistema local.

Extração de Dados: Dependendo do formato do relatório (PDF, Excel, CSV), você precisará usar bibliotecas específicas para extrair os dados. Para PDFs, a biblioteca "PyPDF2" pode ser útil. Para Excel, a biblioteca "pandas" é uma escolha popular.

Transformação de Dados: Após extrair os dados, você precisará transformá-los para o formato desejado. Isso pode incluir limpeza de dados, reestruturação e conversão de tipos de dados.

Carregamento de Dados: Finalmente, carregue os dados transformados em um local de armazenamento de sua escolha, como um banco de dados SQL, um arquivo CSV ou qualquer outro formato adequado. (no meu caso GitHub)

### Comparar as duas bases de dados
#### [United Nations University World Institute for Development Economics Research](https://www.wider.unu.edu/)
com (ONU com OCDE)
####  [Organisation for Economic Co-operation and Development](https://stats.oecd.org/)

https://stats.oecd.org/

In [ ]:
import pandas as pd

# Carregar o arquivo CSV em um DataFrame
path = "/content/OECD_DataGeneric.csv"
df0 = pd.read_csv(path)

# Selecionar as colunas relevantes
columns = ['SUB', 'Subject', 'LOCATION', 'Country', 'YEA', 'Year', 'Unit Code', 'Unit', 'Value']
df0 = df0[columns]
df0.info()

In [ ]:
# Exibir o DataFrame resultante
df0

In [ ]:
df0['Subject'].unique() # verificando os 'temas' dentro de Subject

In [6]:
# Selecionando os campos (Income... está dentro de Subject)
sub_selected01 = "Income inequality: S80/S20 disposable income quintile share"
sub_selected02 = "Country" #escolhendo um país base para comparação
sub_selected03 = "Year"
sub_selected04 = "Value"

#subset_df0 = df0[df0['Subject'] == sub_selected01][['Subject', sub_selected02, sub_selected03, sub_selected04]] #ok
subset_df0 = df0[(df0['Subject'] == sub_selected01)&(df0[sub_selected02] == 'United Kingdom')][['Subject', sub_selected02, sub_selected03, sub_selected04]] #ok

# Exibir a visão parcial dos dados
#print(subset_df)
tab1 = subset_df0

In [ ]:
df0['Subject'].value_counts() # verificação de rotina

https://www.wider.unu.edu/database/world-income-inequality-database-wiid

Quintile, decile, percentile group shares The quintile group shares express the share of total income going to each fifth of the population ordered according to the size of their incomes. In the WIID, these shares are expressed as percentages of total income. The first quintile group includes **the poorest 20 per cent of the population**, while the fifth quintile includes **the richest 20 per cent**. Deciles divide the population into ten groups and percentiles into one hundred groups.

In [3]:
import pandas as pd
df = pd.read_excel('/content/WIID_30JUN2022_0.xlsx')

In [ ]:
list(df.columns) #verificando as colunas

In [ ]:
df['country'].unique() #escolhendo um país base

In [7]:
# Selecionando os dados por colunas específicas
sub_selected1 = "ratio_top20bottom20"
sub_selected2 = "country" #base de comparação
sub_selected3 = "year"
#sub_selected4 = "quality_score" #ainda não sei pra que serve
subset_df = df[df[sub_selected2] == 'United Kingdom'][[sub_selected1, sub_selected2, sub_selected3]] #ok
#subset_df = df[[sub_selected1, sub_selected2, sub_selected3, sub_selected4]] #ok
#subset_df = df[[sub_selected1, sub_selected2, sub_selected3]] #ok

# Exibir a visão parcial dos dados
#print(subset_df)
tab2 = subset_df.groupby(['year', 'country']).mean() #ok #uma vez que os dados de ratio são diversos, foi calcuda a média
#subset_df

In [8]:
print(tab1, tab2) #somente para visualizar

                                                Subject         Country  Year  \
6804  Income inequality: S80/S20 disposable income q...  United Kingdom  2006   
6805  Income inequality: S80/S20 disposable income q...  United Kingdom  2007   
6806  Income inequality: S80/S20 disposable income q...  United Kingdom  2008   
6807  Income inequality: S80/S20 disposable income q...  United Kingdom  2009   
6808  Income inequality: S80/S20 disposable income q...  United Kingdom  2010   
6809  Income inequality: S80/S20 disposable income q...  United Kingdom  2011   
6810  Income inequality: S80/S20 disposable income q...  United Kingdom  2012   

      Value  
6804    6.0  
6805    6.2  
6806    6.1  
6807    6.1  
6808    5.6  
6809    5.6  
6810    5.9                        ratio_top20bottom20
year country                            
1867 United Kingdom                  NaN
1938 United Kingdom                  NaN
1949 United Kingdom             5.750000
1954 United Kingdom             8.

In [32]:
# Criando um DataFrame vazio para armazenar os resultados da comparação
compare_tabs = pd.DataFrame(columns=['Year', 'Country', 'Percentual_Difference', 'ratio_top20bottom20'])

# Iterar sobre os anos em tab2 e realizar a comparação
for year in tab2.index.get_level_values('year'):
    # Verificar se o ano está presente em tab1
    if year in tab1['Year'].values:
        # Selecionar os valores correspondentes de tab1 e tab2
        value_tab1 = tab1[tab1['Year'] == year]['Value'].values[0]
        value_tab2 = tab2.loc[year, 'ratio_top20bottom20'].values[0] #loc - nomes linha/coluna

        # Calcular a diferença percentual
        percentual_difference = ((value_tab2 - value_tab1) / value_tab1) * 100 #tab1 como base do cálculo

        # Adicionar os resultados à tabela de comparação
        row = {'Year': year, 'Country': 'United Kingdom', 'Percentual_Difference': percentual_difference, 'ratio_top20bottom20': value_tab2}
        compare_tabs = compare_tabs.append(row, ignore_index=True)

# Ordenar o DataFrame pelo ano
compare_tabs = compare_tabs.sort_values(by='Year')

# Formatar os valores percentuais com duas casas decimais
compare_tabs['Percentual_Difference'] = compare_tabs['Percentual_Difference'].apply(lambda x: f'{x:.2f}')

# Exibir os resultados
#print(compare_tabs)
compare_tabs

<ipython-input-32-b42df7526a59>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare_tabs = compare_tabs.append(row, ignore_index=True)
<ipython-input-32-b42df7526a59>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare_tabs = compare_tabs.append(row, ignore_index=True)
<ipython-input-32-b42df7526a59>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare_tabs = compare_tabs.append(row, ignore_index=True)
<ipython-input-32-b42df7526a59>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare_tabs = compare_tabs.append(row, ignore_index=True)
<ipython-input-32-b42df7526a59>:17: FutureWarning: The frame.append method is de

,Year,Country,Percentual_Difference,ratio_top20bottom20
0,2006,United Kingdom,12.71,6.762500
1,2007,United Kingdom,9.86,6.811500
2,2008,United Kingdom,11.47,6.799500
3,2009,United Kingdom,8.58,6.623500
4,2010,United Kingdom,18.46,6.634000
5,2011,United Kingdom,15.94,6.492632
6,2012,United Kingdom,4.32,6.154737


In [23]:
#del(compare_tabs)